In [1]:
import gurobipy as gp
from gurobipy import GRB

A company wants to invest $1 billion in each of the four cities, but it doesn't have enough money to do that. Optimization problems have restrictions such that optimal solutions often need to trade off between different objectives.

**Variables**
- Let $X_{A}$, $X_{B}$, $X_{C}$, and $X_{D}$ be the money invested in Atlanta, Boston, Charlotte, and Detroit


In [2]:
### ________ Model ________
model = gp.Model("invest")

### ________ Variables ________
# Define a set of cities
cities = {"Atlanta", "Boston", "Charlotte", "Detroit"}
x = model.addVars(cities, lb=2, ub=8, name="x")

Set parameter Username


**Constraints**

The company's investment budget is $20 million.
$$X_A + X_B + X_C + X_D \le 20$$

The company might believe that there could be regional differences between the two northern cities of Boston and Detroit, and the two southern cities of Atlanta and Charlotte, so they want to hedge a little and invest no more than $12 million in either region.
$$X_B + X_D\le 12$$
$$X_A + X_C\le 12$$

The company might not want to invest more than $8 million and less than $2 million in any city.
$$2 \le X_A\le 8$$
$$2 \le X_B\le 8$$
$$2 \le X_C\le 8$$
$$2 \le X_D\le 8$$



In [3]:
# Define subset of cities
north = {"Boston", "Detroit"}
south = {"Atlanta", "Charlotte"}
# Make sure that 'north' and 'south' are subsets of 'cities'
assert north.issubset(cities)
assert south.issubset(cities)

### ________ Constraints ________
# Budget of $20 million
model.addConstr(x.sum() <= 20, name="Budget")
# Regional limit of $12 million
model.addConstr(gp.quicksum(x.select(north)) <= 20, name="NorthBudget")
model.addConstr(gp.quicksum(x.select(south)) <= 20, name="SouthBudget")
# Max and min investment amount of $8 million and $2 million
# Method 1: include bound when declare variables with 'ub' and 'lb'
# Method 2: add upper and lower bound contrainsts
# model.addConstrs((x[i] <= 8 for i in cities), name="MaxInvest")
# model.addConstrs((x[i] >= 2 for i in cities), name="MinInvest")

<gurobi.Constr *Awaiting Model Update*>

**Objectives**

In this investment example, the objective function might measure the expected increase in value of an investments based on the company's data scientists predictive models. The company is predicting an 11% increase in prices in Atlanta, 2% in Boston, 4% in Charlotte, and 3% in Detroit.
$$\max \left\{ 0.11X_A + 0.02X_B + 0.04X_C + 0.03X_D \right\}  $$

In [5]:
### ________ Objective ________
priceIncrease = {"Atlanta": 0.11, "Boston": 0.02, "Charlotte": 0.04, "Detroit": 0.03}
model.setObjective(x.prod(priceIncrease), GRB.MAXIMIZE)

### ________ Optimize ________
model.optimize()
print(f"Predicted value increase = ${round(model.ObjVal, 2)} million")
for city in cities:
    print(f"Invest ${round(x[city].X, 1)} million in {city}")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12850HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 3 rows, 4 columns and 8 nonzeros
Model fingerprint: 0x07314702
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 1e-01]
  Bounds range     [2e+00, 8e+00]
  RHS range        [2e+01, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 4 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7200000e+00   3.000000e+00   0.000000e+00      0s
       1    1.3000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.300000000e+00
Predicted value increase = $1.3 million
Invest $8.0 million in Atlanta
Invest $8.0 million in Charlotte
Invest $2.0 million